In [1]:
%load_ext autoreload
%autoreload 2

# Dependencies

In [2]:
import datasets_handler
import evaluation_metrics
from zeroberto import ZeroBERTo
import gc
import pandas as pd
from datasets import load_dataset, Dataset
from sklearn.preprocessing import LabelEncoder

google_colab = False

/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:184: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def splitDocuments(docs: pd.Series()) -> list():
0it [00:00, ?it/s]


In [3]:
if google_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    # dataset_path = '/content/drive/Othercomputers/My MacBook Pro/Datasets/folhauol/folhauol_clean_df_articles.csv'
    # dataset_path = '/content/drive/MyDrive/Maquina de London/clean_df_articles_folhauol.csv'
    zeroshot_data_path = './'
else:
    dataset_path = None
    zeroshot_data_path = None

# Data

In [4]:
which_dataset = 'ag_news' 
max_inferences = 5000
random_state = 333
le = LabelEncoder()

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset,path=dataset_path)
classes_list = list(raw_data[class_col].unique())
le.fit(raw_data[class_col])
raw_data = evaluation_metrics.Encoder(raw_data,labelEncoder=le,columnsToEncode=[class_col])
data_to_label = raw_data.sample(max_inferences,random_state=random_state).sort_index()


Found cached dataset ag_news (/Users/alealcoforado/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

# Parameters

In [12]:
zeroshot_method = "dotproduct"  

zeroshot_config = {
    'similarity_model' : 'sentence-transformers/stsb-xlm-r-multilingual', # model for semantic similarity
    'setfit_model' : 'sentence-transformers/stsb-xlm-r-multilingual',# 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2' # model for contrastive learning and classification
    'dataset':which_dataset, ### which dataset we are using
    'class_col':class_col, ### which column of the dataset has the classes
    'data_col':data_col, ### which column of the dataset has the data
    'split':"zeroshot", ### fewshot (uses real training data) or zeroshot (uses previously selected training data)
    'labeling_method':zeroshot_method, ### how the first labeling should be executed
    'max_inferences':max_inferences, ### sample size from full dataset (number of records)
    'classes':classes_list, ### classes from the dataset
    'template': "Esse texto está relacionado a {}.", ### prompt that will be used for semantic search
    'random_state':random_state, ### random state for reproducibility
    'top_n': 8, ### how many best predictions from zeroshot should be recovered
    'training_examples': 8, ### how many examples per class should be used for training
    "batch_size" : 4, ### contrastive training batch size
    "num_pairs" : 10, ### number of text pairs that will be generated in contrastive learning
    "num_epochs" : 1, ### number of epochs training will be conducted
    # 'keep_body_frozen_setfit': True,     # Unfreeze head and freeze body -> head-only training
    'keep_body_frozen_setfit' : False, # Unfreeze head and body -> end-to-end training
    'label_encoder' : le, ### label encoder to use for encoding categorial data into int
    'average_sentence_embeddings':False, ### whether documents should be split by . and their embeddings be a mean of sentences embeddings
    'st_train_epochs': 10, ### sentence-transformer TSDAE fine-tuning --- do not use
    'st_train_batch_size': 20, ### sentence-transformer TSDAE fine-tuning --- do not use
}



# Model

In [13]:
model = ZeroBERTo(classes_list=zeroshot_config['classes'],labeling_dataset=data_to_label,
                embeddingModel=zeroshot_config['similarity_model'],config=zeroshot_config)
       

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


1
queries
3
4


# Labeling

In [14]:
%%time
# if (model.config['split']=='zeroshot'): ###  selecting training data
#     model.runLabeling()
#     model.evaluateLabeling()
#     model.getLabelingMetrics()
#     model.saveLabelingResults(local_path=zeroshot_data_path)
model.config['exec_time'] = '2023_04_10__17_26_04' #### comment above and use this if labeling is already done

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 5.96 µs


Constrative Learning Data

In [15]:
df_train,df_test = datasets_handler.splitDataset(model.labeling_dataset,model.config) 
train_dataset,test_dataset = datasets_handler.buildDatasetDict(df_train,df_test)


Unnamed: 0                                                   2280775
index                                                        1920775
text               Business digest NEW YORK - Investors sent stoc...
label                                                             48
prediction_code                                                   48
top_probability                                             9.995022
dtype: object


In [16]:
df_train

text  \
prediction_code Unnamed: 0                                                      
0               61101       Business digest NEW YORK - Investors sent stoc...   
                20800       ADV: The New York Times Home Delivery Uncover ...   
                52254       DOJ, SEC charge Ex-Peregrine execs over accoun...   
                5409        Chain Store Sales Rise in the Latest Week  NEW...   
                116379      Business confidence dips in Japan Japan indust...   
                11298       Audit Body: Big 4 Firms Missed Problems  NEW Y...   
                118613      Regulators Adopt Tighter Rules on Accounting f...   
                126829      Dollar Steady Ahead of Jobs Report  LONDON (Re...   
1               106306      Mutating Bots May Save Lives Several research ...   
                80884       Cisco, MIT look from lab to market Cisco Syste...   
                66357       Rise of the Robots: Segway Platform Gives Mech...   
                120998      AMD dual-core demo pips Intel, IBM AMD has dem...   
                26465       No News is Open-Source Solaris News Steven Vau...   
                67418       Photo: NEC's new SX-8 supercomputer The SX-8 i...   
                121955      IBM fits PCs with new hardware-based security ...   
                117637      Intel Acquires Chip Designers From HP (AP) AP ...   
2               74272       Eagles' Westbrook Could Miss Some Time  PHILAD...   
                3708        American fouls out OLYMPIA, Greece - When this...   
                38516       Check this out Two 2-0 teams, Terrell Owens an...   
                80075       Jags QB Leftwich sidelined with sprained knee ...   
                74325       Eagles #39; Westbrook Could Miss Some Time PHI...   
                108954      Another good BCS debacle?  quot;The purpose of...   
                21396       Athletics not worth the price Big-time college...   
                92834       Loss Behind Them, Eagles Dominate Again (AP) A...   
3               61642       SOMALIA: New government gets African diplomati...   
                79118       Adams Urges Stronger UN Response to Darfur Cri...   
                3238        Egypt announces 'breakthrough' on Mideast peac...   
                101418      ASEAN eyes European model Vientiane (AsiaNews/...   
                64857       Israel, Egypt Move Toward Warm Relations (AP) ...   
                119738      Annan Seeks EU Support for U.N. Reforms (AP) A...   
                29201       Kofi Annan: Globally Popular Mainstay of the U...   
                106780      EU takes over Bosnia mission The EU is due to ...   

                            class_code  
prediction_code Unnamed: 0              
0               61101                0  
                20800                0  
                52254                0  
                5409                 0  
                116379               0  
                11298                0  
                118613               0  
                126829               0  
1               106306               1  
                80884                1  
                66357                1  
                120998               1  
                26465                1  
                67418                1  
                121955               1  
                117637               1  
2               74272                2  
                3708                 2  
                38516                2  
                80075                2  
                74325                2  
                108954               2  
                21396                2  
                92834                2  
3               61642                3  
                79118                3  
                3238                 3  
                101418               3  
                64857                3  
                119738               3  
      

# Training

In [17]:
### overrides config for looping second part of the model
# model.config['num_epochs'] = 5
# model.config['num_pairs'] = 20
# # model.config['keep_body_frozen_setfit'] = False

### build trainer for contrastive learning
model.buildTrainer(train_dataset)

### train
model.contrastive_train()
gc.collect()

### predict labeling dataset with new classifier
# model.getPredictions()

model.predict(test_dataset['text'])
# 5000 sentences --- 4min40s ---sts-xlm-rr
# 5000 sentences ---  ---ricardo-filho/bert-portuguese


Applying column mapping to training dataset
***** Running training *****
  Num examples = 640
  Num epochs = 1
  Total optimization steps = 160
  Total train batch size = 4


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/160 [00:00<?, ?it/s]

Predicting 5000 documents


array([0, 0, 0, ..., 1, 0, 1])

In [11]:
setfit_all_metrics = evaluation_metrics.get_metrics(model.y_pred, test_dataset["class_code"])
print(setfit_all_metrics)

setfit_exec_time  = evaluation_metrics.saveResults(model.config,setfit_all_metrics,None)
print(model.config)

{'weighted': [{'accuracy': 0.7274}, {'precision': 0.7593511627168696}, {'recall': 0.7274}, {'f1': 0.7144892571190165}], 'macro': [{'accuracy': 0.7274}, {'precision': 0.7591079912108177}, {'recall': 0.7255513261086622}, {'f1': 0.71286686494949}]}
metrics_setfit_2023_04_10__17_53_12.csv
config_setfit_2023_04_10__17_53_12.csv
{'similarity_model': 'sentence-transformers/stsb-xlm-r-multilingual', 'setfit_model': 'sentence-transformers/stsb-xlm-r-multilingual', 'dataset': 'ag_news', 'class_col': 'class', 'data_col': 'text', 'split': 'zeroshot', 'labeling_method': 'dotproduct', 'max_inferences': 5000, 'classes': ['business', 'science and technology', 'sports', 'world'], 'template': 'Esse texto está relacionado a {}.', 'random_state': 333, 'top_n': 4, 'training_examples': 4, 'batch_size': 4, 'num_pairs': 10, 'num_epochs': 1, 'keep_body_frozen_setfit': False, 'label_encoder': LabelEncoder(), 'average_sentence_embeddings': False, 'st_train_epochs': 10, 'st_train_batch_size': 20, 'exec_time': '20